# TPSC : Self energy


Nearest neighbour $t$, next nearest neighbour $t'=-0.175$, and next next nearest neighbour hopping $t''=0.05$, interaction $U = 5.75t$, and inverse temperature $\beta = 20$.

The following points are not clear to me:

- In the TPSC there is a density $n$. Should it be the same as the density in $G_0$? Or should it match the density of the interacting $G$?

- How do I define $G_0$ away from half-filling. Is it

    $G_0(k) = \frac{1}{z + \mu - \epsilon_k - n U}$

In [ ]:
%reload_ext cpp2py.magic

In [ ]:
%%cpp2py -C pytriqs
#include <triqs/gfs.hpp>
using namespace triqs::gfs;

// The type of a Green function : (k,omega) -> Complex number
using g_k_w_type = gf_view<cartesian_product<brillouin_zone, imfreq>, scalar_valued>;
using g_r_t_type = gf<cartesian_product<cyclic_lattice, imtime>, scalar_valued>;

g_k_w_type bubble(g_k_w_type g0) {
    
    // Fourier Transformation of k, \omega to obtain g(r,t)
    auto grt = make_gf_from_fourier<0,1>(g0);
    
    // The mesh of gtr is a cartesian product mt x mr. We decompose it.
    auto [mr, mt] = grt.mesh();
    
    // The inverse temperature from the mesh
    double beta = mt.domain().beta;
    
    // A new mesh for chi, with a bosonic statistics, but same size as mt.
    auto mtb = gf_mesh<imtime>{beta, Boson, mt.size()};
    
    // Build chi (r, tau) with this new mesh.
    auto chi0 = g_r_t_type{{mr, mtb}};

    // we fill chi : chi(tau, r) = g(beta - tau, -r) * g(tau, r)
    for (auto const &r : mr)      
        for (auto const &t : mtb) 
            chi0[r, t] = grt(-r, beta - t) * grt(r, t); 

    // Fourier transform back to k, \omega space and return
    return make_gf_from_fourier<0,1>(chi0);
}

In [ ]:
%%cpp2py -C pytriqs
#include <triqs/gfs.hpp>
using namespace triqs::gfs;

// The type of a Green function : (k,omega) -> Complex number
using g_k_w_type = gf_view<cartesian_product<brillouin_zone, imfreq>, scalar_valued>;

g_k_w_type bubble2(g_k_w_type chi, g_k_w_type g0) {
    
    // Fourier Transformation of k, \omega to obtain g(r,t)
    auto chirt = make_gf_from_fourier<0,1>(chi);
    auto g0rt = make_gf_from_fourier<0,1>(g0);
    
    auto sigma = g0rt;
    sigma() = 0;
    
    // The mesh of gtr is a cartesian product mt x mr. We decompose it.
    auto [mr, mt] = g0rt.mesh();
    
    // The inverse temperature from the mesh
    double beta = mt.domain().beta;

    // we fill sigma : sigma(r, tau) = chi(r, tau) * g0(r, tau)
    // I am not sure this is correct .......
    for (auto const &r : mr)
        for (auto const &t : mt)
            sigma[r, t] = chirt(r,t) * g0rt(r,t); 

    // Fourier transform back to k, \omega space and return
    return make_gf_from_fourier<0,1>(sigma);
}

## Imports

In [ ]:
from pytriqs.plot.mpl_interface import plt, oplot, oplotr, oploti
import numpy as np
from pytriqs.lattice import BravaisLattice, BrillouinZone
from pytriqs.gf import MeshBrillouinZone, MeshImFreq, Gf, MeshProduct, Idx
from pytriqs.gf import inverse, MeshImTime, MeshCyclicLattice, Fourier, InverseFourier
from pytriqs.archive import HDFArchive
from scipy.optimize import fsolve, brentq

## Parameters

In [ ]:
beta = 20.
mu = 0.0 # Should we adjust to get correct density?

t = 1.0
tp = -0.175 * t
tpp = 0.05 * t
U = 5.75 * t

n_k = 128
n_w = 8*1024

BL = BravaisLattice([(1, 0, 0), (0, 1, 0)])
BZ = BrillouinZone(BL)
kmesh = MeshBrillouinZone(BZ, n_k = n_k)

## Functions

In [ ]:
# TODO: check the dispersion here !!!
def eps(kx,ky):
    return -2 * t * (np.cos(kx) + np.cos(ky)) \
           -4 * tp * np.cos(kx) * np.cos(ky) \
           -2 * tpp * (np.cos(2*kx) + np.cos(2*ky))

def get_g0(beta):
    wmesh = MeshImFreq(beta=beta, S='Fermion', n_max=n_w)
    w = np.tensordot(np.ones(n_k*n_k), list(wmesh.values()), 0)
    k = np.tensordot(list(kmesh.values()), np.ones(2*n_w), 0)
    kx, ky = k[:,0,:], k[:,1,:]

    g0 = Gf(mesh = MeshProduct(kmesh, wmesh), target_shape = [])
    g0.data[:,:] = 1 / (w - eps(kx,ky))
    return g0

def get_chi0(g0):    
    return bubble(g0)

def chi_rpa(chi0_wk, U):
    chi_rpa = chi0_kw.copy()
    chi_rpa = chi0_kw * inverse(1 - U * chi0_kw)
    return chi_rpa

def trace_chi_kw(chi_kw):
    kmesh, wmesh = chi_kw.mesh.components
    trace = chi_kw.data.sum() / len(kmesh) / wmesh.beta
    return trace.real

def Usp_root_problem(Usp, chi0, n, U):
    tr_chi_sp = trace_chi_kw(chi_rpa(chi0, U=Usp))
    diff = 2*tr_chi_sp + 0.5 * Usp/U * n**2 - n
    return diff

def Uch_root_problem(Uch, chi0, n, U, docc):
    tr_chi_ch = trace_chi_kw(chi_rpa(chi0, U=-Uch[0]))
    diff = 2*tr_chi_ch - 2 * docc - n + n**2
    return diff

def solve_Usp_and_Uch(chi0, U, n, Usp0=0.1, Uch0=0.1):
    Uc = 1/chi0([np.pi,np.pi,0],0).real
    Usp = brentq(Usp_root_problem, 0, Uc-1e-6, args=(chi0, n, U), xtol=1e-2)
    docc = 0.25 * Usp / U * n**2
    Uch = fsolve(Uch_root_problem, Uch0, args=(chi0, n, U, docc), xtol=1e-2)[0]
    return Usp, Uch, docc, Uc

## Plot the MDC

In [ ]:
k = np.linspace(-np.pi, np.pi, n_k+1, endpoint=True)
kx, ky = np.meshgrid(k, k)

g0 = get_g0(beta)
spectral = lambda kx, ky: (-1/np.pi)*g0([kx,ky,0],0).imag
fs = lambda kx, ky: (1/g0([kx,ky,0],0)).real

plt.figure(figsize=(4,4))
plt.pcolor(kx, ky, np.vectorize(spectral)(kx,ky))
plt.contour(kx, ky, np.vectorize(fs)(kx,ky), levels=[0], colors='white')
plt.axes().set_aspect('equal')

# Cosmetics
plt.xticks([-np.pi, 0, np.pi],[r"$-\pi$", r"0", r"$\pi$"])    
plt.yticks([-np.pi, 0, np.pi],[r"$-\pi$", r"0", r"$\pi$"])
plt.xlabel(r"$k_x$")
plt.ylabel(r"$k_y$")
plt.title("$-\mathrm{Im}G_k(i\omega_0)$")

## TPSC calculation


In [ ]:
# Get g0 and compute its non-interacting density
g0_kw = get_g0(beta)
g_sum = g0_kw[Idx(0,0,0),:].copy()
g_sum.zero()
for k in kmesh:
    g_sum += g0_kw[k,:]
g_sum /= n_k * n_k

n = 2*g_sum.density().real
print 'density =', n

In [ ]:
# Get Lindhard chi
chi0_kw = get_chi0(g0_kw)

# Solve TPSC
Usp, Uch = 1., 1.
Usp, Uch, docc, Uc = solve_Usp_and_Uch(chi0_kw, U, n, Usp0=Usp, Uch0=Uch)

print 'Usp, Uch, docc, Uc =', Usp, Uch, docc, Uc

# Store TPSC spin and charge susceptibility
chi_sp_kw = chi_rpa(chi0_kw, Usp)
chi_ch_kw = chi_rpa(chi0_kw, -Uch)

In [ ]:
# Plot the spin susceptibility

k = np.linspace(0, 2*np.pi, n_k+1, endpoint=True)
kx, ky = np.meshgrid(k, k)

chis = lambda kx, ky: chi_sp_kw([kx,ky,0],0).real
chic = lambda kx, ky: chi_ch_kw([kx,ky,0],0).real

plt.figure(figsize=(10,4))

plt.subplot(121)
plt.pcolor(kx,ky,np.vectorize(chis)(kx,ky))
plt.xticks([0, np.pi, 2*np.pi],[r"$0$", r"$\pi$", r"$2\pi$"])    
plt.yticks([0, np.pi, 2*np.pi],[r"$0$", r"$\pi$", r"$2\pi$"])
plt.xlabel(r"$k_x$")
plt.ylabel(r"$k_y$")
plt.xlim(0, 2*np.pi)
plt.ylim(0, 2*np.pi)
plt.colorbar()

plt.subplot(122)
plt.pcolor(kx,ky,np.vectorize(chic)(kx,ky))
plt.xticks([0, np.pi, 2*np.pi],[r"$0$", r"$\pi$", r"$2\pi$"])    
plt.yticks([0, np.pi, 2*np.pi],[r"$0$", r"$\pi$", r"$2\pi$"])
plt.xlabel(r"$k_x$")
plt.ylabel(r"$k_y$")
plt.xlim(0, 2*np.pi)
plt.ylim(0, 2*np.pi)
plt.colorbar()

CHECK: The data in `chi_sp_kw` and `chi_ch_kw` looks clean

In [ ]:
oplot(chi_sp_kw[Idx(n_k/2,n_k/2,0),:])
oplot(chi_ch_kw[Idx(n_k/2,n_k/2,0),:])

oplot(chi_sp_kw[Idx(0,0,0),:])
oplot(chi_ch_kw[Idx(0,0,0),:])

oplot(chi_sp_kw[Idx(0,n_k/2,0),:])
oplot(chi_ch_kw[Idx(0,n_k/2,0),:])

plt.xlim(-10,10)

## Now we can compute the self-energy

In [ ]:
# compute self energy
g_chi_sp_kw = 2*bubble2(chi_sp_kw, g0_kw)
g_chi_ch_kw = 2*bubble2(chi_ch_kw, g0_kw)

sigma_kw = U/8. * ( 3.*Usp*g_chi_sp_kw + Uch*g_chi_ch_kw )

CHECK: The data in `sigma_kw` looks clean

In [ ]:
oplot(sigma_kw[Idx(0,0,0),:])
oplot(sigma_kw[Idx(n_k/2,n_k/2,0),:])
oplot(sigma_kw[Idx(0,n_k/2,0),:])
plt.xlim(-50,50)

## MDC of the self-energy

In [ ]:
#k = np.linspace(-np.pi, np.pi, n_k+1, endpoint=True)
k = np.linspace(-np.pi, np.pi, 200, endpoint=True)

kx, ky = np.meshgrid(k, k)

sigma = lambda kx, ky: sigma_kw([kx,ky,0],0).imag

plt.figure(figsize=(4,4))
plt.pcolor(kx,ky,np.vectorize(sigma)(kx,ky))
plt.axes().set_aspect('equal')

# Cosmetics
plt.xticks([-np.pi, 0, np.pi],[r"$-\pi$", r"0", r"$\pi$"])    
plt.yticks([-np.pi, 0, np.pi],[r"$-\pi$", r"0", r"$\pi$"])
plt.xlabel(r"$k_x$")
plt.ylabel(r"$k_y$")
plt.xlim(-np.pi, np.pi)
plt.ylim(-np.pi, np.pi)
plt.colorbar()
#plt.title("Momentum distribution curve (MDC) at the Fermi level")

## MDC of the Green's function

What is the density of the Green's function?

In [ ]:
#k = np.linspace(-np.pi, np.pi, n_k+1, endpoint=True)
k = np.linspace(0, np.pi, 200, endpoint=True)
kx, ky = np.meshgrid(k, k)

# TODO: Take care of Hartree shift ???
green = lambda kx, ky: (-1/np.pi)*(1 / (-eps(kx,ky) - sigma_kw([kx,ky,0],0) + 0.03j)).imag

plt.figure(figsize=(4,4))
plt.pcolor(kx, ky, np.vectorize(green)(kx,ky))
plt.axes().set_aspect('equal')
plt.plot([1.48,0.92,0.54],[1.48,2.1,np.pi],'ow',lw=4,alpha=0.8)

# Cosmetics
plt.xticks([0, np.pi],["0", r"$\pi$"])    
plt.yticks([0, np.pi],["0", r"$\pi$"])
plt.xlabel(r"$k_x$")
plt.ylabel(r"$k_y$")
plt.xlim(0, np.pi)
plt.ylim(0, np.pi)
plt.colorbar()
plt.title("Momentum distribution curve (MDC) at the Fermi level")

## (Optional) Plot some EDC's

For this part to work we need:

- To have `set_from_pade` for scalar valued Gf
- To have the evaluator for `<brillouin_zone, refreq>` Gf's
- Or even better the interpolator `(k, all)`

In [ ]:
# Analytic continuation of Sigma_wk to w = 0 + i*epsilon

from pytriqs.gf import MeshReFreq

rwmesh = MeshReFreq(omega_min=-0.4, omega_max=0.1, n_max=100)
sigma_krw = Gf(mesh=MeshProduct(kmesh,rwmesh), target_shape=[1,1])

# This is because set_from_pade does not work with scalar yet
from pytriqs.gf import GfImFreq, SemiCircular
tmp = GfImFreq(indices=[0], beta=beta, n_points=8*1024)

om = np.linspace(-0.4,0.1,100)
for kx,ky in [(1.48,1.48), (0.92,2.1), (0.54,np.pi)]:
    
    # Ugly !!!
    ki = Idx(int(kx*n_k/(2*np.pi)), int(ky*n_k/(2*np.pi)), 0)

    # Ugly !!!
    tmp.data[:,0,0] = sigma_kw[ki,:].data[:]
    sigma_krw[ki,:].set_from_pade(tmp, n_points=100)
    #sigma_krw[ki,:].set_from_pade(sigma_kw[ki,:], n_points=100)

    gom = lambda omega: ((-1/np.pi) / \
                         (omega - eps(kx,ky) - sigma_krw[ki,:](omega)[0,0] + 0.03j)).imag
    
    plt.plot(om, np.vectorize(gom)(om), label="k = (%.2f,%.2f)"%(kx,ky))
    
plt.legend(loc='best')
plt.title("EDC at node, antinode, hotspot")
plt.xlabel(r"$\omega$")
plt.ylabel(r"$A(k,\omega)$")